# 🪟 Interactive Sliding Window Technique Tutorial

## 📚 How to Use This Notebook:

### 1️⃣ Installation (Run this first!):
```bash
pip install matplotlib ipywidgets numpy
```

### 2️⃣ What You'll Learn:
- ✅ Fixed-size sliding window
- ✅ Variable-size (dynamic) sliding window
- ✅ Two-pointer technique
- ✅ Real-world applications

### 3️⃣ Difficulty Progression:
- 🟢 **Basic**: Fixed window sum/max
- 🟡 **Intermediate**: Variable window with conditions
- 🔴 **Advanced**: Complex constraints & optimizations

### 4️⃣ Color Guide:
- 🟦 **Blue** = Outside window
- 🟨 **Yellow** = Inside window
- 🟩 **Green** = Optimal/result
- 🟥 **Red** = Current element being processed

---

### 📦 Library Imports - Step by Step

1. **Import matplotlib** → Creates visual plots and animations
2. **Import ipywidgets** → Enables interactive controls
3. **Import numpy** → Numerical array operations
4. **Configure matplotlib inline** → Display plots in notebook
5. **Set figure size** → 14x6 inches for clear visibility

In [47]:
# Import required libraries
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np

# Configure matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['animation.html'] = 'jshtml'

print("✅ All libraries imported successfully!")
print("\n🚀 Ready to explore Sliding Window!")

✅ All libraries imported successfully!

🚀 Ready to explore Sliding Window!


### 🔧 FixedWindowVisualizer Class - Step by Step Logic

**`find_max_sum()` method:**
1. **Initialize first window** → Sum first `k` elements: `window_sum = sum(arr[:k])`
2. **Set initial max** → `max_sum = window_sum`, `max_start = 0`
3. **Record first step** → Store window position [0:k-1] and sum
4. **Slide window from index k to n-1:**
   - **Remove leftmost** → Subtract `arr[i - k]` (element leaving window)
   - **Add rightmost** → Add `arr[i]` (element entering window)
   - **Formula**: `window_sum = window_sum - arr[i-k] + arr[i]`
   - **Update max if better** → Compare and update `max_sum`
   - **Record step** → Store current window state

**Why O(n) instead of O(n*k)?**
- Instead of recalculating sum of k elements each time
- We reuse previous sum: subtract old element, add new element
- Each element is added once and removed once

**`animate()` method:**
1. **Call find_max_sum()** → Generate all steps
2. **Create figure** → Two subplots (array view + info panel)
3. **For each frame:**
   - Draw array with colored rectangles (yellow = in window)
   - Add red bracket around current window
   - Display window sum, max sum, best position

---
## 🟢 Part 1: Basic Fixed-Size Sliding Window

### What is a Sliding Window?
A **sliding window** is a sub-array/sub-string that moves through the data structure.

### Problem: Find Maximum Sum of K Consecutive Elements
Given an array, find the maximum sum of any `k` consecutive elements.

### 🎯 Example Walkthrough: arr=[2,1,5,1,3,2,8,1,3], k=3

**Step-by-step execution:**
1. **Window [0:2]** → [2,1,5] → sum=8, max=8
2. **Window [1:3]** → [1,5,1] → sum=8-2+1=7, max=8
3. **Window [2:4]** → [5,1,3] → sum=7-1+3=9, max=9 ✓
4. **Window [3:5]** → [1,3,2] → sum=9-5+2=6, max=9
5. **Window [4:6]** → [3,2,8] → sum=6-1+8=13, max=13 ✓
6. **Window [5:7]** → [2,8,1] → sum=13-3+1=11, max=13
7. **Window [6:8]** → [8,1,3] → sum=11-2+3=12, max=13

**Result**: Maximum sum = 13 at window [4:6]

In [48]:
class FixedWindowVisualizer:
    def __init__(self, arr, k):
        self.arr = arr
        self.k = k
        self.n = len(arr)
        self.steps = []
        
    def find_max_sum(self):
        self.steps = []
        window_sum = sum(self.arr[:self.k])
        max_sum = window_sum
        max_start = 0
        
        self.steps.append({
            'window_start': 0, 'window_end': self.k - 1,
            'window_sum': window_sum, 'max_sum': max_sum,
            'max_start': max_start,
            'action': f'Initial window [0:{self.k-1}], sum={window_sum}'
        })
        
        for i in range(self.k, self.n):
            window_sum = window_sum - self.arr[i - self.k] + self.arr[i]
            if window_sum > max_sum:
                max_sum = window_sum
                max_start = i - self.k + 1
                action = f'New max! sum={window_sum}'
            else:
                action = f'sum={window_sum}, max still {max_sum}'
            
            self.steps.append({
                'window_start': i - self.k + 1, 'window_end': i,
                'window_sum': window_sum, 'max_sum': max_sum,
                'max_start': max_start, 'action': action
            })
        return max_sum, max_start
    
    def animate(self, interval=800):
        self.find_max_sum()
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
        
        def update(frame):
            ax1.clear(); ax2.clear()
            step = self.steps[frame]
            ws, we = step['window_start'], step['window_end']
            ms, me = step['max_start'], step['max_start'] + self.k - 1

            for i, val in enumerate(self.arr):
                # Green for best result window, Yellow for current window, Blue for outside
                if ms <= i <= me:
                    color = '#90EE90'  # Green for best result
                elif ws <= i <= we:
                    color = '#FFD700'  # Yellow for current window
                else:
                    color = '#87CEEB'  # Blue for outside
                rect = patches.Rectangle((i, 0), 0.9, 1, facecolor=color, edgecolor='black', linewidth=2)
                ax1.add_patch(rect)
                ax1.text(i + 0.45, 0.5, str(val), ha='center', va='center', fontsize=14, fontweight='bold')
                ax1.text(i + 0.45, -0.3, str(i), ha='center', va='center', fontsize=10, color='gray')

            # Red bracket for current window
            bracket = patches.FancyBboxPatch((ws - 0.1, -0.1), we - ws + 1.1, 1.2,
                                           boxstyle='round,pad=0.05', facecolor='none', edgecolor='red', linewidth=3)
            ax1.add_patch(bracket)

            # Green bracket for best result window
            if ms != ws or me != we:
                best_bracket = patches.FancyBboxPatch((ms - 0.1, -0.1), me - ms + 1.1, 1.2,
                                               boxstyle='round,pad=0.05', facecolor='none', edgecolor='green', linewidth=2, linestyle='--')
                ax1.add_patch(best_bracket)

            ax1.set_xlim(-0.5, self.n + 0.5); ax1.set_ylim(-0.8, 1.5); ax1.axis('off')
            ax1.set_title(f'Step {frame + 1}/{len(self.steps)}: {step["action"]}', fontsize=14, fontweight='bold')
            
            info_text = f"Window: [{ws}:{we}]\nWindow Sum: {step['window_sum']}\nMax Sum: {step['max_sum']}\nBest: [{step['max_start']}:{step['max_start'] + self.k - 1}]"
            ax2.text(0.5, 0.5, info_text, ha='center', va='center', fontsize=14, transform=ax2.transAxes,
                    bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
            ax2.axis('off')
        
        anim = FuncAnimation(fig, update, frames=len(self.steps), interval=interval, repeat=True)
        plt.close()
        return HTML(anim.to_jshtml())

print("✅ Fixed Window Visualizer created!")

✅ Fixed Window Visualizer created!


### 🔧 VariableWindowVisualizer Class - Step by Step Logic

**`find_min_subarray()` method - Two Pointer Approach:**

1. **Initialize pointers** → `left = 0`, `right` will iterate
2. **Initialize tracking** → `window_sum = 0`, `min_len = infinity`
3. **For each right pointer (0 to n-1):**
   - **EXPAND**: Add `arr[right]` to `window_sum`
   - **CONTRACT while sum >= target:**
     - Update `min_len` if current window is smaller
     - Subtract `arr[left]` from `window_sum`
     - Move `left` pointer right (`left += 1`)

**Key Insight - Why this works:**
- **Expand** (right++) → Grow window until condition met
- **Contract** (left++) → Shrink to find minimum valid window
- Each element visited at most twice (once by right, once by left)
- Time: O(n), not O(n²)

**Two Phases in each step:**
- **'expand' phase** → Adding element, window may be invalid
- **'contract' phase** → Window valid (sum >= target), try to shrink

### 🎮 Try it: Maximum Sum of K Elements

### 🎯 Example Walkthrough: arr=[2,3,1,2,4,3], target=7

**Step-by-step execution:**
1. **right=0**: sum=2, <7 → expand
2. **right=1**: sum=5, <7 → expand
3. **right=2**: sum=6, <7 → expand
4. **right=3**: sum=8, ≥7 → contract! len=4, then sum=6
5. **right=4**: sum=10, ≥7 → contract! len=4, sum=7, ≥7 → contract! len=3, sum=6
6. **right=5**: sum=9, ≥7 → contract! len=3, sum=7, ≥7 → contract! len=2 ✓

**Result**: Minimum length = 2 (subarray [4,3])

In [49]:
arr = [2, 1, 5, 1, 3, 10, 8, 1, 3,4]
k = 3
viz = FixedWindowVisualizer(arr, k)
viz.animate(interval=1000)

### 🔧 SubstringVisualizer Class - Step by Step Logic

**`find_longest()` method - HashMap + Sliding Window:**

1. **Initialize** → `char_index = {}` (stores last seen position of each char)
2. **Initialize pointers** → `left = 0`, `max_len = 0`
3. **For each right pointer (0 to n-1):**
   - **Get current char** → `char = s[right]`
   - **Check for duplicate:**
     - If `char` in `char_index` AND `char_index[char] >= left`:
       - Move `left` to `char_index[char] + 1` (skip past duplicate)
   - **Update char position** → `char_index[char] = right`
   - **Update max length** → `max_len = max(max_len, right - left + 1)`

**Key Insight - Why HashMap:**
- Instantly know if character exists in current window
- Instantly know where to move left pointer
- No need to scan window for duplicates

**Why `char_index[char] >= left` check?**
- Character might exist in HashMap but be outside current window
- Example: "abba" → at 'a' (index 3), old 'a' at 0 is already excluded

### 🎯 Example Walkthrough: s="abcabcbb"

**Step-by-step execution:**
1. **right=0, 'a'**: window="a", len=1, max=1
2. **right=1, 'b'**: window="ab", len=2, max=2
3. **right=2, 'c'**: window="abc", len=3, max=3 ✓
4. **right=3, 'a'**: duplicate! left=1, window="bca", len=3, max=3
5. **right=4, 'b'**: duplicate! left=2, window="cab", len=3, max=3
6. **right=5, 'c'**: duplicate! left=3, window="abc", len=3, max=3
7. **right=6, 'b'**: duplicate! left=5, window="cb", len=2, max=3
8. **right=7, 'b'**: duplicate! left=7, window="b", len=1, max=3

**Result**: Longest substring = "abc" (length 3)

---
## 🟡 Part 2: Variable-Size Sliding Window

### Problem: Smallest Subarray with Sum ≥ Target

In [50]:
class VariableWindowVisualizer:
    def __init__(self, arr, target):
        self.arr = arr
        self.target = target
        self.n = len(arr)
        self.steps = []
        
    def find_min_subarray(self):
        self.steps = []
        left = 0
        window_sum = 0
        min_len = float('inf')
        result_window = None
        
        for right in range(self.n):
            window_sum += self.arr[right]
            self.steps.append({'left': left, 'right': right, 'window_sum': window_sum,
                'min_len': min_len if min_len != float('inf') else 'N/A',
                'result': result_window, 'action': f'Expand: add arr[{right}]={self.arr[right]}', 'phase': 'expand'})
            
            while window_sum >= self.target:
                if right - left + 1 < min_len:
                    min_len = right - left + 1
                    result_window = (left, right)
                self.steps.append({'left': left, 'right': right, 'window_sum': window_sum,
                    'min_len': min_len, 'result': result_window,
                    'action': f'✓ sum={window_sum} >= {self.target}', 'phase': 'contract'})
                window_sum -= self.arr[left]
                left += 1
        return min_len if min_len != float('inf') else 0
    
    def animate(self, interval=800):
        result = self.find_min_subarray()
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
        
        def update(frame):
            ax1.clear(); ax2.clear()
            step = self.steps[frame]
            left, right = step['left'], step['right']
            result_window = step['result']
            
            for i, val in enumerate(self.arr):
                # Green for best result window
                if result_window and result_window[0] <= i <= result_window[1]:
                    color = '#90EE90'  # Green for best result
                elif left <= i <= right:
                    color = '#FFD700' if step['phase'] == 'expand' else '#FFA500'  # Yellow/Orange for current
                else:
                    color = '#87CEEB'  # Blue for outside
                rect = patches.Rectangle((i, 0), 0.9, 1, facecolor=color, edgecolor='black', linewidth=2)
                ax1.add_patch(rect)
                ax1.text(i + 0.45, 0.5, str(val), ha='center', va='center', fontsize=14, fontweight='bold')
            
            ax1.annotate('L', xy=(left + 0.45, 0), xytext=(left + 0.45, -0.6), fontsize=12, ha='center', color='red', fontweight='bold', arrowprops=dict(arrowstyle='->', color='red'))
            ax1.annotate('R', xy=(right + 0.45, 1), xytext=(right + 0.45, 1.5), fontsize=12, ha='center', color='blue', fontweight='bold', arrowprops=dict(arrowstyle='->', color='blue'))
            
            # Green bracket for best result window
            if result_window:
                rs, re = result_window
                best_bracket = patches.FancyBboxPatch((rs - 0.1, -0.1), re - rs + 1.1, 1.2,
                                               boxstyle='round,pad=0.05', facecolor='none', edgecolor='green', linewidth=3)
                ax1.add_patch(best_bracket)
            
            ax1.set_xlim(-0.5, self.n + 0.5); ax1.set_ylim(-1, 2); ax1.axis('off')
            ax1.set_title(f'Step {frame + 1}: {step["action"]}', fontsize=14, fontweight='bold')
            
            result_str = f"[{step['result'][0]}:{step['result'][1]}]" if step['result'] else 'None'
            info_text = f"Target: {self.target}\nWindow: [{left}:{right}]\nSum: {step['window_sum']}\nMin Length: {step['min_len']}\nBest: {result_str}"
            ax2.text(0.5, 0.5, info_text, ha='center', va='center', fontsize=12, transform=ax2.transAxes,
                    bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
            ax2.axis('off')
        
        anim = FuncAnimation(fig, update, frames=len(self.steps), interval=interval, repeat=True)
        plt.close()
        print(f"\n✅ Minimum length subarray with sum >= {self.target}: {result}")
        return HTML(anim.to_jshtml())

print("✅ Variable Window Visualizer created!")

✅ Variable Window Visualizer created!


### 🎮 Try it: Minimum Subarray Sum

In [51]:
arr = [2, 3, 1, 2, 4, 3]
target = 7
viz = VariableWindowVisualizer(arr, target)
viz.animate(interval=1000)


✅ Minimum length subarray with sum >= 7: 2


---
## 🟡 Part 3: Longest Substring Without Repeating Characters

In [52]:
class SubstringVisualizer:
    def __init__(self, s):
        self.s = s
        self.n = len(s)
        self.steps = []
        
    def find_longest(self):
        self.steps = []
        char_index = {}
        left = 0
        max_len = 0
        result = (0, 0)
        
        for right in range(self.n):
            char = self.s[right]
            if char in char_index and char_index[char] >= left:
                left = char_index[char] + 1
                action = f"Duplicate '{char}'! Move left to {left}"
            else:
                action = f"Add '{char}'"
            
            char_index[char] = right
            if right - left + 1 > max_len:
                max_len = right - left + 1
                result = (left, right)
                action += f" → New max: {max_len}"
            
            self.steps.append({'left': left, 'right': right, 'char_index': dict(char_index),
                'max_len': max_len, 'result': result, 'action': action})
        return max_len, self.s[result[0]:result[1]+1]
    
    def animate(self, interval=800):
        max_len, substr = self.find_longest()
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
        
        def update(frame):
            ax1.clear(); ax2.clear()
            step = self.steps[frame]
            left, right = step['left'], step['right']
            rs, re = step['result']
            
            for i, char in enumerate(self.s):
                # Green for best result window
                if rs <= i <= re:
                    color = '#90EE90'  # Green for best result
                elif left <= i <= right:
                    color = '#FFD700' if i == right else '#FFA500'  # Yellow for current
                else:
                    color = '#87CEEB'  # Blue for outside
                rect = patches.Rectangle((i, 0), 0.9, 1, facecolor=color, edgecolor='black', linewidth=2)
                ax1.add_patch(rect)
                ax1.text(i + 0.45, 0.5, char, ha='center', va='center', fontsize=16, fontweight='bold')
            
            # Green bracket for best result
            best_bracket = patches.FancyBboxPatch((rs - 0.1, -0.1), re - rs + 1.1, 1.2,
                                           boxstyle='round,pad=0.05', facecolor='none', edgecolor='green', linewidth=3)
            ax1.add_patch(best_bracket)
            
            ax1.set_xlim(-0.5, self.n + 0.5); ax1.set_ylim(-0.8, 1.5); ax1.axis('off')
            ax1.set_title(f'Step {frame + 1}: {step["action"]}', fontsize=14, fontweight='bold')
            
            info_text = f"Window: \"{self.s[left:right+1]}\" (len={right-left+1})\nMax: {step['max_len']}\nBest: \"{self.s[step['result'][0]:step['result'][1]+1]}\""
            ax2.text(0.5, 0.5, info_text, ha='center', va='center', fontsize=12, transform=ax2.transAxes,
                    bbox=dict(boxstyle='round', facecolor='lavender', alpha=0.8))
            ax2.axis('off')
        
        anim = FuncAnimation(fig, update, frames=len(self.steps), interval=interval, repeat=True)
        plt.close()
        print(f"\n✅ Longest substring: \"{substr}\" (len={max_len})")
        return HTML(anim.to_jshtml())

print("✅ Substring Visualizer created!")

✅ Substring Visualizer created!


In [53]:
s = "abcabcbb"
viz = SubstringVisualizer(s)
viz.animate(interval=1000)


✅ Longest substring: "abc" (len=3)


---
## 🔴 Part 4: Sliding Window Maximum (Monotonic Deque)

In [54]:
from collections import deque

class WindowMaxVisualizer:
    def __init__(self, arr, k):
        self.arr = arr
        self.k = k
        self.n = len(arr)
        self.steps = []
        
    def find_maximums(self):
        self.steps = []
        dq = deque()
        result = []
        
        for i in range(self.n):
            while dq and dq[0] <= i - self.k:
                dq.popleft()
            while dq and self.arr[dq[-1]] < self.arr[i]:
                dq.pop()
            dq.append(i)
            
            if i >= self.k - 1:
                result.append(self.arr[dq[0]])
                action = f'Window max = {self.arr[dq[0]]}'
            else:
                action = f'Building window... ({i+1}/{self.k})'
            
            self.steps.append({'index': i, 'window_start': max(0, i - self.k + 1), 'window_end': i,
                'deque': list(dq), 'deque_values': [self.arr[j] for j in dq],
                'result': result.copy(), 'action': action, 'max_idx': dq[0] if dq else None})
        return result
    
    def animate(self, interval=800):
        result = self.find_maximums()
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [2, 1, 1]})
        
        def update(frame):
            ax1.clear(); ax2.clear(); ax3.clear()
            step = self.steps[frame]
            ws, we = step['window_start'], step['window_end']
            dq_indices = step['deque']
            max_idx = step['max_idx']
            
            for i, val in enumerate(self.arr):
                if ws <= i <= we:
                    if max_idx is not None and i == max_idx:
                        color = '#90EE90'  # Green for current maximum
                    elif i in dq_indices:
                        color = '#FFD700'  # Yellow for elements in deque
                    else:
                        color = '#FFA500'  # Orange for window elements not in deque
                else:
                    color = '#87CEEB'  # Blue for outside window
                rect = patches.Rectangle((i, 0), 0.9, 1, facecolor=color, edgecolor='black', linewidth=2)
                ax1.add_patch(rect)
                ax1.text(i + 0.45, 0.5, str(val), ha='center', va='center', fontsize=14, fontweight='bold')
            
            # Green bracket around current window maximum
            if max_idx is not None and we >= self.k - 1:
                max_bracket = patches.FancyBboxPatch((max_idx - 0.1, -0.1), 1.1, 1.2,
                                               boxstyle='round,pad=0.05', facecolor='none', edgecolor='green', linewidth=3)
                ax1.add_patch(max_bracket)
            
            ax1.set_xlim(-0.5, self.n + 0.5); ax1.set_ylim(-0.3, 1.5); ax1.axis('off')
            ax1.set_title(f'Step {frame + 1}: {step["action"]}', fontsize=14, fontweight='bold')
            
            ax2.set_xlim(-0.5, 6); ax2.set_ylim(-0.5, 1.5)
            ax2.text(-0.3, 0.5, 'Deque:', fontsize=12, fontweight='bold', va='center')
            for i, (idx, val) in enumerate(zip(dq_indices, step['deque_values'])):
                # First element (max) is green
                color = '#90EE90' if i == 0 else '#DDA0DD'
                rect = patches.Rectangle((i + 1, 0), 0.9, 1, facecolor=color, edgecolor='black', linewidth=2)
                ax2.add_patch(rect)
                ax2.text(i + 1.45, 0.5, f'{val}', ha='center', va='center', fontsize=12, fontweight='bold')
            ax2.axis('off')
            
            ax3.text(0.1, 0.5, f'Results: {step["result"]}', fontsize=14, transform=ax3.transAxes,
                    bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
            ax3.axis('off')
        
        anim = FuncAnimation(fig, update, frames=len(self.steps), interval=interval, repeat=True)
        plt.close()
        print(f"\n✅ Maximum in each window: {result}")
        return HTML(anim.to_jshtml())

print("✅ Window Maximum Visualizer created!")

✅ Window Maximum Visualizer created!


In [55]:
arr = [1, 3, -1, -3, 5, 3, 6, 7]
k = 3
viz = WindowMaxVisualizer(arr, k)
viz.animate(interval=1000)


✅ Maximum in each window: [3, 3, 5, 5, 6, 7]


---
## 📝 Code Reference

In [56]:
# ===== FIXED WINDOW =====
def max_sum_k(arr, k):
    window_sum = sum(arr[:k])
    max_sum = window_sum
    for i in range(k, len(arr)):
        window_sum = window_sum - arr[i - k] + arr[i]
        max_sum = max(max_sum, window_sum)
    return max_sum

# ===== VARIABLE WINDOW =====
def min_subarray_sum(arr, target):
    left = window_sum = 0
    min_len = float('inf')
    for right in range(len(arr)):
        window_sum += arr[right]
        while window_sum >= target:
            min_len = min(min_len, right - left + 1)
            window_sum -= arr[left]
            left += 1
    return min_len if min_len != float('inf') else 0

# ===== LONGEST UNIQUE SUBSTRING =====
def longest_unique(s):
    char_index = {}
    left = max_len = 0
    for right, char in enumerate(s):
        if char in char_index and char_index[char] >= left:
            left = char_index[char] + 1
        char_index[char] = right
        max_len = max(max_len, right - left + 1)
    return max_len

# ===== SLIDING WINDOW MAXIMUM =====
def sliding_max(arr, k):
    from collections import deque
    dq = deque()
    result = []
    for i in range(len(arr)):
        while dq and dq[0] <= i - k: dq.popleft()
        while dq and arr[dq[-1]] < arr[i]: dq.pop()
        dq.append(i)
        if i >= k - 1: result.append(arr[dq[0]])
    return result

print("Tests:")
print(f"Max sum k=3: {max_sum_k([2, 1, 5, 1, 3, 2], 3)}")
print(f"Min subarray >= 7: {min_subarray_sum([2, 3, 1, 2, 4, 3], 7)}")
print(f"Longest unique: {longest_unique('abcabcbb')}")
print(f"Sliding max k=3: {sliding_max([1, 3, -1, -3, 5, 3, 6, 7], 3)}")

Tests:
Max sum k=3: 9
Min subarray >= 7: 2
Longest unique: 3
Sliding max k=3: [3, 3, 5, 5, 6, 7]


---
## 🎓 Summary

### Types of Sliding Window
| Type | Size | Example |
|------|------|--------|
| Fixed | K | Max sum of K elements |
| Variable | Dynamic | Min subarray >= target |
| HashMap | Dynamic | Longest unique substring |
| Deque | Fixed | Window maximum |

### Template
```python
left = 0
for right in range(n):
    # Add arr[right] to window
    while invalid:
        # Remove arr[left]
        left += 1
    # Update result
```

### Complexity
- **Time**: O(n)
- **Space**: O(1) or O(k)

### Practice
- LeetCode #3: Longest Substring Without Repeating
- LeetCode #209: Minimum Size Subarray Sum
- LeetCode #239: Sliding Window Maximum
- LeetCode #76: Minimum Window Substring